In [1]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import cv2 
from tqdm import tqdm
import os

In [3]:
RANDOM_STATE = 42  #setting the random variable value for reobataining the variable parameters

In [4]:
image_data_train = os.path.join('dataset/'+'eyes_only/train')  #defining the path where dataset is stored
image_data_test = os.path.join('dataset/'+'eyes_only/test')

In [5]:
from keras.preprocessing.image import ImageDataGenerator

In [6]:
train_gen = ImageDataGenerator(rescale=1/255,
                                zoom_range=0.2,
                                shear_range=0.4,
                                horizontal_flip=True,
                                rotation_range=40,
                                fill_mode='nearest',
                                brightness_range=[0.2,1])

test_gen = ImageDataGenerator(rescale=1/255)      #data augmentation

In [7]:
train_set = train_gen.flow_from_directory(image_data_train,
                                          target_size=(128,128),
                                          shuffle=True,
                                          batch_size=25,
                                          class_mode='binary')
test_set = test_gen.flow_from_directory(image_data_test,
                                        class_mode='binary',
                                        shuffle=False,
                                        target_size=(128,128),
                                        batch_size=15)   
# Date Preprocessing                                    

Found 1234 images belonging to 2 classes.
Found 218 images belonging to 2 classes.


In [17]:
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D,Flatten,BatchNormalization,Dropout,Dense
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.applications import VGG16,ResNet50V2,xception,inception_v3
from livelossplot import PlotLossesKeras
import pydot
from keras.utils.vis_utils import plot_model

In [18]:
def call_model():
    model_cnn = Sequential()
    model_cnn.add(Conv2D(64,(3,3),input_shape=(128,128,3),padding='same',activation='relu'))
    model_cnn.add(MaxPooling2D(pool_size=(3,3)))
    model_cnn.add(Conv2D(64,(3,3),activation='relu',padding='valid'))
    model_cnn.add(MaxPooling2D(pool_size=(2,2)))
    model_cnn.add(BatchNormalization())
    model_cnn.add(Conv2D(32,(3,3),activation='relu',padding='valid'))
    model_cnn.add(MaxPooling2D(2,2))
    model_cnn.add(Conv2D(32,(3,3),activation='relu',padding='valid'))
    model_cnn.add(MaxPooling2D(2,2))
    model_cnn.add(BatchNormalization())
    model_cnn.add(Flatten())

    model_cnn.add(Dense(512,activation='relu'))
    model_cnn.add(Dense(64,activation='relu'))
    model_cnn.add(Dense(1,activation='sigmoid'))


    model_cnn.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')
    

    return model_cnn

    #defining a function that will return model (sequential) 

In [23]:
model = call_model()
#plot_model(model,to_file='model_cnn_1.png',show_shapes=True,show_layer_names=True)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 128, 128, 64)      1792      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 42, 42, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 40, 40, 64)        36928     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 20, 20, 64)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 20, 20, 64)        256       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 18, 18, 32)        18464     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 9, 9, 32)         

In [25]:
h = model.fit(train_set,
              steps_per_epoch=train_set.n//25,
              epochs=10,
              validation_steps=test_set.n//15,
              validation_data=test_set)

Epoch 1/10
49/49 [==============================] - 12s 250ms/step - loss: 0.2128 - accuracy: 0.9074 - val_loss: 1.2124 - val_accuracy: 0.5571
Epoch 2/10
49/49 [==============================] - 8s 164ms/step - loss: 0.1227 - accuracy: 0.9537 - val_loss: 1.4494 - val_accuracy: 0.6749
Epoch 3/10
49/49 [==============================] - 8s 157ms/step - loss: 0.1110 - accuracy: 0.9537 - val_loss: 1.5804 - val_accuracy: 0.6798
Epoch 4/10
49/49 [==============================] - 8s 154ms/step - loss: 0.0793 - accuracy: 0.9719 - val_loss: 1.7686 - val_accuracy: 0.6798
Epoch 5/10
49/49 [==============================] - 8s 155ms/step - loss: 0.0971 - accuracy: 0.9620 - val_loss: 0.2032 - val_accuracy: 0.8867
Epoch 6/10
49/49 [==============================] - 7s 153ms/step - loss: 0.0788 - accuracy: 0.9628 - val_loss: 1.2546 - val_accuracy: 0.8128
Epoch 7/10
49/49 [==============================] - 8s 155ms/step - loss: 0.0741 - accuracy: 0.9686 - val_loss: 0.7602 - val_accuracy: 0.9458
Epoch

In [26]:
model.save('models/model_cnn_1.h5') #saving the trained model